从修饰总表中对有修饰与无修饰进行pair对组合。表格的形式要求为：
sequence target communal-modification measure unit modification1 modification2 mic1 mic2 fold-change sig(0,-1,1)


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
os.chdir(r'E:\文件\大学课程\大四上\毕业设计\数据分析\seventhweek')
# all_df=pd.read_excel('all-peptides.xlsx')

In [2]:
# 在固定某一端修饰、measure、unit一致时，提取另一端所有修饰数据并将其创建成为一个字典
def get_minvalue(value1,value2):
    if value1>value2:
        return value1
    else:
        return value2


def get_activity_array(file,change_ter, stick_ter,stick_modification,measure,unit):
    df = pd.read_excel(file, header=0)
    df = df.fillna(value='none')
    df1 = df[(df[stick_ter] == stick_modification) & (df['other_modification'] == 'none') & (df['unit']==unit) & (df['measure']==measure)]
    dict_matrix = {}
    for index, row in df1.iterrows():
        seq_tar = '&'.join([row['sequence'], row['target'],stick_ter,stick_modification,measure,unit])            #共有部分先用&进行连接，方便后续直接进行分列操作
        if seq_tar not in dict_matrix.keys():
            dict_matrix[seq_tar] = {}
            dict_matrix[seq_tar][row[change_ter]] = float(row['value'])
        else:
            value1=dict_matrix[seq_tar][row[change_ter]] = float(row['value'])
            value2=float(row['value'])
            dict_matrix[seq_tar][row[change_ter]] = get_minvalue(value1,value2)     #可能存在多个样本其他字段内字符相同但是value值不同，称之为重复样本，那么此时会优先考虑保留活性较强的样本，即测算方法数值较小的那个
    return dict_matrix

In [3]:
# # 数据中间查看
# data1=get_activity_array(file='peptides_revise_value.xlsx',stick_ter='nterminal',change_ter='cterminal',stick_modification='none',unit='uM',measure='MIC')

In [4]:
# #48561
# # 查看数据
# no_change_dict=pd.DataFrame(data1)
# no_change_dict.T

In [5]:
#循环获得的字典，对字典内的活性数值进行删除修改
# 如果none存在，则判断是否还存在其他修饰，则直接计算有修饰与无修饰时的fold-change（计算方法为：fold-change=log2(none/modification)
def RESERVE_TWO_ACTIVITIES(data1):
    data=copy.deepcopy(data1)
    for key1 in list(data.keys()):
        if 'none' in data[key1].keys() and len(list(data[key1].keys())) >= 2:
            pass
        else:
            del data[key1]
    return data

In [6]:
# # 查看数据
# dffff=pd.DataFrame(RESERVE_TWO_ACTIVITIES(data1))
# dffff.T
# #在数据还在字典的时候，循环字典里的字典的key（即小字典里的key），如果none和有修饰同时存在，那么有修饰则赋值为fold-change，如果不符合以上条件，则将大字典的key删掉

In [7]:
# 把获取的字典导出成为目标数据表
# 将获取的数据表转换成为目标格式
# sequence target communal-modification measure unit modification1 modification2 mic1 mic2 fold-change sig(0,-1,1)
def GOAL_DATAFRAME(data):
    dict_to_excel=pd.DataFrame.from_dict(data)
    dict_to_excel=dict_to_excel.transpose()         #将数据表进行转置
    columns=['sequence&target&stick_terminal&communal-modification&measure&unit','modification1','modification2','activity1','activity2','fold-change','abs(none-modification)']
    new_index=0
    new_activity_df=pd.DataFrame(columns=columns)
    for modification in list(dict_to_excel.columns):
        if modification=='none':
            pass
        else:
            modification_not_none=dict_to_excel[dict_to_excel[modification].notnull() & dict_to_excel['none'].notnull()]
            for index,row in modification_not_none.iterrows():
                fold_change=np.log2(row['none']/row[modification])
                abs_activity=abs(row['none']-row[modification])
                new_row=[index,'none',modification,row['none'],row[modification],fold_change,abs_activity]
                new_activity_df.loc[new_index]=new_row
                new_index+=1

    #将合并的列拆分并重新命名列
    #拆分得列会变成一个新的数据表，所以需要pd.concat函数将两个数据表按列进行合并，排在前面的放在合并数据表的前面
    new_activity_df = pd.concat([new_activity_df['sequence&target&stick_terminal&communal-modification&measure&unit'].str.split('&', expand=True),new_activity_df], axis=1).drop('sequence&target&stick_terminal&communal-modification&measure&unit', axis=1)   #进行分列操作并将原本拆分的列删除掉
    new_column='sequence&target&stick_terminal&communal-modification&measure&unit'.split('&')

    #给拆分后的列的列名重命名，注意要使用inplace=True函数代替掉原来的列名，不然达不到修改的目的
    rename_colunm={}
    for i in range(len(new_column)):
        rename_colunm[i]=new_column[i]
    new_activity_df.rename(columns=rename_colunm,inplace=True)
    return new_activity_df

In [8]:
# 循环列表格式
# file_name,change_ter, stick_ter,stick_modification,measure,unit
# 创建循环列表
def CYCLE():
    terminals=['n_terminal','c_terminal']
    stick_modifications=['none','ACT','AMD']
    measures=['MIC','MBC','IC50','LC','MIC50','MIC90','MFC','EC50','LD50','IC50 I','MEC','IC50 REP','LC50','IC50 F','IC50 E']
    units=['uM','ug/ml']
    cycle=[]
    # cycle=[[file,'cterminal','nterminal','none','MIC']]
    for terminal in terminals:
        sub_cycle=['all-peptides.xlsx']
        stick_terminal=terminal
        if stick_terminal=='n_terminal':
            change_terminal='c_terminal'
        else:
            change_terminal='n_terminal'
        sub_cycle.append(change_terminal)
        sub_cycle.append(stick_terminal)
        for modification in stick_modifications:
            if modification=='none':
                sub_cycle.append(modification)
            elif stick_terminal=='n_terminal' and modification=='ACT':
                sub_cycle.append(modification)
            elif stick_terminal=='c_terminal' and modification=='AMD':
                sub_cycle.append(modification)
            for measure in measures:
                sub_cycle.append(measure)
                for unit in units:
                    sub_cycle.append(unit)
                    new_sub_cycle=copy.deepcopy(sub_cycle)
                    if len(new_sub_cycle)==6:
                        cycle.append(new_sub_cycle)
                    else:
                        break
                    # print(sub_cycle)
                    sub_cycle.pop(-1)
                sub_cycle.pop(-1)
            sub_cycle.pop(-1)
        sub_cycle=[]
    return cycle

In [9]:
# 把所有分类写入文件输出
# file_name,change_ter, stick_ter,stick_modification,measure,unit
clasification_data=pd.DataFrame(columns=['file-name','chang-terminal','stick-terminal','stick-modification','measure','unit'])
for i in range(len(CYCLE())):
    clasification_data.loc[i]=CYCLE()[i]
clasification_data.to_excel('classification.xlsx',index=False)

In [10]:
#写一个脚本把之前所有的函数都合起来一起用，实现循环，把所有的分类给合并成一个总的数据表，要保证所有函数的输入参数都是一样的才能一次性进行循环

# 读入文件，筛选数据表,并将数据转换成为字典
index=0
mdf=pd.DataFrame()
for single_cycle in CYCLE():
    data1=get_activity_array(single_cycle[0],single_cycle[1],single_cycle[2],single_cycle[3],single_cycle[4],single_cycle[5])
    data=RESERVE_TWO_ACTIVITIES(data1)
    single_dataframe=GOAL_DATAFRAME(data)
    mdf=pd.concat([mdf,single_dataframe])


In [11]:
mdf.to_excel('all-peptides-pairs.xlsx',index=False)

# 循环存在的修饰情况：
需要比较的修饰有：
N端:ACT,脂肪链（C1,C2,C3,....)
C端:AMD
stick-terminal stick-modification none modification
nterminal none none ACT
nterminal ACT none ACT
cterminal none none ACT
cterminal AMD none ACT
cterminal none none C
cterminal AMD none C


UNIT CHANGE: ug/ml uM